# Uczenie maszynowe <br> Zadanie domowe nr 2: Klasyfikacja z zastosowaniem sieci neuronowych - z elementami konkursu
Politechnika Gdańska, Wydział ETI, Katedra Inżynierii Biomedycznej

# Cel:
Opracować procedurę złożoną z przetwarzania wstępnego oraz klasyfikacji danych z wykorzystaniem sieci neuronowych (uwaga - jedynie MLP - bez CNN, itp.) w celu uzyskania jak najlepszych wyników klasyfikacji dla zadanego zbioru danych.

# Problem:
Dla zbioru CIFAR-10 należy wybrać przykłady dla wskazanych 3 klas. Następnie opracować procedurę w celu uzyskania najlepszych wyników klasyfikacji mierzonych z użyciem miary F1-score, F1 = 2(recall*precision)/(recall+precision). Do wyznaczenia wartości miary należy zastosować funkcję f1_score z pakietu scikit-learn (from sklearn.metrics import f1_score).

# Dane studentów
14.06.2021 <br>
Inżynieria biomedyczna, Sztuczna inteligencja gr. 1 <br>
Patrycja Gładkowska 171951 <br>
Szymon Sadowski 165298

In [1]:
# Download data and select classes 'plane', 'cat', 'truck'

import random
from keras.datasets import cifar10
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


indexes = [0, 3, 9]

idx_train = np.array([idx for idx in range(y_train.shape[0]) if y_train[idx] in indexes])
idx_test = np.array([idx for idx in range(y_test.shape[0]) if y_test[idx] in indexes])

x_train = X_train[idx_train]
y_train = y_train[idx_train]

x_test = X_test[idx_test]
y_test = y_test[idx_test]

In [2]:
# Preprocessing
epochs = 100
batch_size = 32
nb_classes = 10

# Reshape data
X_train = x_train.reshape(-1, 32 * 32 * 3)
X_test = x_test.reshape(-1, 32 * 32 * 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [3]:
# Classification

# MLP
model = Sequential()
model.add(Dense(1024, input_shape=(3072, )))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# training
history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, Y_test))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              3146752   
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2

469/469 [==============================] - 5s 11ms/step - loss: 0.4854 - accuracy: 0.8067 - val_loss: 0.5600 - val_accuracy: 0.7847
Epoch 46/100
469/469 [==============================] - 5s 11ms/step - loss: 0.4887 - accuracy: 0.8062 - val_loss: 0.5546 - val_accuracy: 0.7810
Epoch 47/100
469/469 [==============================] - 5s 11ms/step - loss: 0.4884 - accuracy: 0.8044 - val_loss: 0.5354 - val_accuracy: 0.7920
Epoch 48/100
469/469 [==============================] - 5s 11ms/step - loss: 0.4721 - accuracy: 0.8097 - val_loss: 0.5294 - val_accuracy: 0.7903
Epoch 49/100
469/469 [==============================] - 5s 11ms/step - loss: 0.4836 - accuracy: 0.8023 - val_loss: 0.5668 - val_accuracy: 0.7743
Epoch 50/100
469/469 [==============================] - 5s 11ms/step - loss: 0.4612 - accuracy: 0.8212 - val_loss: 0.5408 - val_accuracy: 0.7757
Epoch 51/100
469/469 [==============================] - 5s 11ms/step - loss: 0.4715 - accuracy: 0.8166 - val_loss: 0.5759 - val_accuracy: 0.767

In [4]:
# Metrics

Y_pred = model.predict(X_test, verbose=2)

loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print(f"Loss:, {loss}")
print(f"Accuracy: {acc}")

print("Confusion matrix:")
conf_matrix = confusion_matrix(Y_test.argmax(axis=1), Y_pred.argmax(axis=1))
print(conf_matrix)

f1_score = f1_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), average="weighted")
print(f"F1 score: {f1_score}")

print("Classification report:")
target_names = ['plane', 'cat', 'truck']
classification_report = classification_report(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), target_names=target_names)
print(classification_report)

94/94 - 0s
Loss:, 0.5531776547431946
Accuracy: 0.7806666493415833
Confusion matrix:
[[676 164 160]
 [ 50 861  89]
 [ 66 129 805]]
F1 score: 0.7792478664660093
Classification report:
              precision    recall  f1-score   support

       plane       0.85      0.68      0.75      1000
         cat       0.75      0.86      0.80      1000
       truck       0.76      0.81      0.78      1000

    accuracy                           0.78      3000
   macro avg       0.79      0.78      0.78      3000
weighted avg       0.79      0.78      0.78      3000



In [5]:
# Extra validation

In [6]:
# enter your data
# X_test = ''
# Y_test = ''

# Y_pred = model.predict(X_test, verbose=2)

# loss, acc = model.evaluate(X_test, Y_test, verbose=0)
# print('Loss:', loss)
# print(f"Accuracy: {acc}")

# print("Confusion matrix:")
# conf_matrix_test = confusion_matrix(Y_test.argmax(axis=1), Y_pred.argmax(axis=1))
# print(conf_matrix_test)

# f1_score_test = f1_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), average="weighted")
# print(f"F1 score: {f1_score_test}")

# print("Classification report:")
# target_names = ['plane', 'cat', 'truck']
# classification_report_test = classification_report(Y_test.argmax(axis=1), Y_pred.argmax(axis=1), target_names=target_names)
# print(classification_report_test)